# Deploy MLflow Model to online endpoint

- Before running this notebook, run the **Chapter 6 Prep-Model Creation & Registration.ipynb** notebook to create and register a model for use

- Models created with MLFlow do not require a scoring script nor an environment


In [36]:
import azure.ai.ml
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)

print(azure.ai.ml._version.VERSION)

1.1.1


In [37]:
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

In [38]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    # This will open a browser page for
    credential = InteractiveBrowserCredential()

In [39]:
#connect to the workspace
try:
    ml_client = MLClient.from_config(credential=credential)
except Exception as ex:
    # NOTE: Update following workspace information if not correctly configure before
    client_config = {
        "subscription_id": subscription_id,
        "resource_group": resource_group,
        "workspace_name": workspace,
    }

    if client_config["subscription_id"].startswith("<"):
        print(
            "please update your <SUBSCRIPTION_ID> <RESOURCE_GROUP> <AML_WORKSPACE_NAME> in notebook cell"
        )
        raise ex
    else:  # write and reload from config file
        import json, os

        config_path = "../.azureml/config.json"
        os.makedirs(os.path.dirname(config_path), exist_ok=True)
        with open(config_path, "w") as fo:
            fo.write(json.dumps(client_config))
        ml_client = MLClient.from_config(credential=credential, path=config_path)
print(ml_client)

Found the config file in: /mnt/batch/tasks/shared/LS_root/mounts/clusters/memasanz1/code/Users/memasanz/ML-Engineering-with-Azure-Machine-Learning-Service/.azureml/config.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7f38d0111c00>,
         subscription_id=b071bca8-0055-43f9-9ff8-ca9a144c2a6f,
         resource_group_name=aml-dev-rg,
         workspace_name=aml-dev)


## Create Online Endpoint

To create an online endpoint, we will leverage the class *ManagedOnlineEndpoint*.  To create the online endpoint we will provide the following configuration:

- name of endpoint
- description
- auth_mode (set to key) or aml_token
- tags - to provide information regarding the endpoint

### Configure endpoint

In [40]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "chp6-mlflow-endpt-" + datetime.datetime.now().strftime("%m%d%H%M%f")
print(len(online_endpoint_name))
# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="titanic online endpoint for mlflow model",
    auth_mode="key",
    tags={"mlflow": "true"},
)

32


## Create endpoint

Using the MLClient created earlier, we will now create the Endpoint in the workspace. This command will start the endpoint creation and return a confirmation response while the endpoint creation continues.


In [41]:
ml_client.begin_create_or_update(endpoint)

## Create deployment
A deployment is a set of resouces used for hosting the inferecing model using the *ManagedOnlineDeployment* class.  
Using the *ManagedOnlineDeployment* class, a developer can configure the following components

- name: name of the deployment
- endpoint_name: name of the endpoint to create the deployment under
- model: the model to use for the deployment
- instance_type: the VM side to use for deployment
- instance_count: the number of instances to use for the deployment
    

# Retrieve Model from registry

In [42]:
import mlflow

In [44]:
experiment = 'chapter6'
current_experiment=dict(mlflow.get_experiment_by_name(experiment))
experiment_id=current_experiment['experiment_id']
print(experiment_id)

df = mlflow.search_runs([experiment_id])

run_id = df['run_id'].tail(1).values[0]
print(run_id)

e5ccf20f-54f7-4511-8ca7-15703445747c
sincere_shirt_95vn1fk2nc


In [45]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

run_model = Model(
    name="chapter6_titanic_model",
    path="azureml://jobs/" + run_id + "/outputs/artifacts/paths/model/",
    description="Model created from run.",
    type=AssetTypes.MLFLOW_MODEL 
)


In [46]:
run_model

Model({'job_name': None, 'is_anonymous': False, 'auto_increment_version': True, 'name': 'chapter6_titanic_model', 'description': 'Model created from run.', 'tags': {}, 'properties': {}, 'id': None, 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/memasanz1/code/Users/memasanz/ML-Engineering-with-Azure-Machine-Learning-Service/chapter 6', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f3887071db0>, 'version': None, 'latest_version': None, 'path': 'azureml://jobs/sincere_shirt_95vn1fk2nc/outputs/artifacts/paths/model/', 'datastore': None, 'utc_time_created': None, 'flavors': None, 'arm_type': 'model_version', 'type': 'mlflow_model'})

## Configure the deployment

- retrieve the experiment id for this run, and the run id to retrieve the model from the registered model list

In [47]:
import time
while ml_client.online_endpoints.get(name=online_endpoint_name).provisioning_state == 'Creating':
    print('Creating')
    time.sleep(10)

print(ml_client.online_endpoints.get(name=online_endpoint_name).provisioning_state)

Succeeded


In [48]:
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=run_model,
    instance_type="Standard_F4s_v2",
    instance_count=1,
)

In [49]:
## Create deployment
print(online_endpoint_name)

chp6-mlflow-endpt-11282304439097


In [50]:
ml_client.online_deployments.begin_create_or_update(blue_deployment)

Check: endpoint chp6-mlflow-endpt-11282304439097 exists
data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


In [51]:
import time
while ml_client.online_deployments.get(name = "blue", endpoint_name = online_endpoint_name).provisioning_state == 'Updating':
    print('Updating..will take about 10 minutes to deploy...')
    time.sleep(5)
    
ml_client.online_deployments.get(name = "blue", endpoint_name = online_endpoint_name).provisioning_state

.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
..Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
..Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
.Updating
..

'Succeeded'

In [53]:
## get status of online deployment
ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=online_endpoint_name, lines=50
)

"Instance status:\nSystemSetup: Succeeded\nUserContainerImagePull: Succeeded\nModelDownload: Succeeded\nUserContainerStart: Succeeded\n\nContainer events:\nKind: Pod, Name: ReadinessProbeFailed, Type: Warning, Time: 2022-11-28T23:12:16.811423Z, Message: Readiness probe failed: HTTP probe failed with statuscode: 502\nKind: Pod, Name: LivenessProbeFailed, Type: Warning, Time: 2022-11-28T23:12:26.207871Z, Message: Liveness probe failed: HTTP probe failed with statuscode: 502\nKind: Pod, Name: ReadinessProbeFailed, Type: Warning, Time: 2022-11-28T23:12:26.811525Z, Message: Readiness probe failed: HTTP probe failed with statuscode: 502\nKind: Pod, Name: LivenessProbeFailed, Type: Warning, Time: 2022-11-28T23:12:36.207687Z, Message: Liveness probe failed: HTTP probe failed with statuscode: 502\nKind: Pod, Name: ReadinessProbeFailed, Type: Warning, Time: 2022-11-28T23:12:36.81158Z, Message: Readiness probe failed: HTTP probe failed with statuscode: 502\nKind: Pod, Name: LivenessProbeFailed, T

In [55]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint)

ResourceExistsError: (Conflict) Conflict
Code: Conflict
Message: Conflict
Exception Details:	(OperationDuplicationConflict) Conflict of operation, another operation on same entity is already running in workspace aml-dev.
	Code: OperationDuplicationConflict
	Message: Conflict of operation, another operation on same entity is already running in workspace aml-dev.

## Checking provisioning state

In [56]:
import time
while ml_client.online_deployments.get(name = "blue", endpoint_name = online_endpoint_name).provisioning_state == 'Updating':
    print('Updating')
    time.sleep(5)
    
ml_client.online_deployments.get(name = "blue", endpoint_name = online_endpoint_name).provisioning_state

'Succeeded'

## Get Endpoint details

In [57]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(endpoint)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

primary_key = ml_client.online_endpoints.get_keys(name = online_endpoint_name).primary_key

print(primary_key)

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://chp6-mlflow-endpt-11282304439097.eastus2.inference.ml.azure.com/score', 'openapi_uri': 'https://chp6-mlflow-endpt-11282304439097.eastus2.inference.ml.azure.com/swagger.json', 'name': 'chp6-mlflow-endpt-11282304439097', 'description': 'titanic online endpoint for mlflow model', 'tags': {'mlflow': 'true'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/b071bca8-0055-43f9-9ff8-ca9a144c2a6f/resourcegroups/aml-dev-rg/providers/microsoft.machinelearningservices/workspaces/aml-dev/onlineendpoints/chp6-mlflow-endpt-11282304439097', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/b071bca8-0055-43f9-9ff8-ca9a144c2a6f/providers/Microsoft.MachineLearningServices/locations/eastus2/mfeOperationsStatus/oe:9634b616-29cb-4345-ae22-4be4e4bfe009:20a39653-046b-4b29-822a-af866773cfb9?api-version=2022-02-01-preview'}, 'id': '/subscriptions/b071bca8-0055-43f9

### Leverage the registered ml table from Chapter 4 to get some data to send to the rest endpoint.

- Previously in Chapter 4, we registered the MLTable: titanic_prepped_mltable_x2, we will retrieve it and convert it to a pandas datafame

In [58]:
import mltable
registered_v1_data_asset = ml_client.data.get(name='titanic_prepped_mltable_x2', version='1')
print(registered_v1_data_asset.path)

tbl = mltable.load(uri=registered_v1_data_asset.path)
df = tbl.to_pandas_dataframe()

azureml://subscriptions/b071bca8-0055-43f9-9ff8-ca9a144c2a6f/resourcegroups/aml-dev-rg/workspaces/aml-dev/datastores/workspaceblobstore/paths/LocalUpload/b3e9d2d76d36b52fc88b17546f0f0460/titanic_prepped_mltable/


In [59]:
columns_to_keep =  ['Embarked', 'Loc', 'Sex','Pclass', 'Age', 'Fare', 'GroupSize']
X_raw           = df[columns_to_keep]
X_raw

,Embarked,Loc,Sex,Pclass,Age,Fare,GroupSize
0,S,X,m,3,22.0,7.25,2
1,C,C,f,1,38.0,71.2833,2
2,S,X,f,3,26.0,7.925,1
3,S,C,f,1,35.0,53.1,2
4,S,X,m,3,35.0,8.05,1
...,...,...,...,...,...,...,...
886,S,X,m,2,27.0,13.0,1
887,S,B,f,1,19.0,30.0,1
888,S,X,f,3,21.5,23.45,4
889,C,C,m,1,26.0,30.0,1


In [60]:
import json
url = endpoint.scoring_uri
api_key = primary_key  # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'blue' }
import requests

def MakePrediction(df):
    strjson = str(df.to_json(orient='records'))
    endpoint_url = url

    request_data =  {
              "input_data": {
                "columns": [
                  "Embarked",
                  "Loc",
                  "Sex",
                  "Pclass",
                  "Age",
                  "Fare",
                  "GroupSize"
                ],
                "data": []
              }
            }

    request_data['input_data']['data'] = json.loads(X_raw.to_json(orient='records'))
    parsed = json.dumps(request_data)
    
    r = requests.post(endpoint_url, headers=headers, data=parsed)
    return (r.json())

results = MakePrediction(X_raw)
print(results)

[0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 